In [30]:
import os
from openai import OpenAI
from pathlib import Path
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from fpdf import FPDF
from PIL import Image
import requests

In [31]:
# Access the API key from the environment variables
api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI()

In [32]:
def text_bot(subject):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1.5,
        messages=[
            {
                "role": "system",
                "content": '''You are a highly creative and professional content generator. 
                Your goal is to produce high-quality, engaging, and unique content tailored to the user's request. 
                Ensure the tone, style, and length match the user's specifications.'''},
            {
                "role": "user",
                "content": f"Create a 120-word blog post about {subject}."
            }
        ]
    )

    return completion.choices[0].message.content

# text_bot('the importance of sustainable energy and its impact on the environment')

In [33]:
def img_bot(subject, save=True):
    response = client.images.generate(
    model="dall-e-2",
    prompt=subject,
    size="1024x1024",
    quality="standard",
    n=1,
)

    if save:
        current_directory = Path().resolve()
        img_path = current_directory / 'img.jpg'
        img_url = response.data[0].url
        image_response = requests.get(img_url, stream=True)

        # Save the image to the current directory
        with open(img_path, "wb") as file:
            for chunk in image_response.iter_content(1024):  # Download in chunks
                file.write(chunk)
            
        return str(img_path)
        
    return img_url

# img_bot("the importance of sustainable energy and its impact on the environment", save=True)

In [34]:
def TTS_bot(text):
    current_directory = Path().resolve()
    speech_file_path = current_directory / "speech.mp3"
    response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=text,
    )
    
    # Save the response content manually
    with open(speech_file_path, "wb") as audio_file:
        audio_file.write(response.content)

    return speech_file_path

# TTS_bot(text_bot('the importance of sustainable energy and its impact on the environment'))

In [35]:
def get_embedding(text, model="text-embedding-ada-002"):
    """
    Fetch the embedding for a given text using OpenAI's embedding model.
    """
    response = client.embeddings.create(
        input="Your text string goes here",
        model="text-embedding-3-small"
    )

    return np.array(response.data[0].embedding)

def generate_topics(input_text, num_topics=5):
    """
    Generate related topics dynamically using GPT.
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that suggests related topics for content creation."},
            {"role": "user", "content": f"Suggest {num_topics} related topics for: {input_text}"}
        ]
    )
    # Extract the topics from the response
    topics = response.choices[0].message.content.split("\n")
    return [topic.strip("- ") for topic in topics if topic.strip()]

def suggest_related_topics(input_text):
    """
    Suggest related topics dynamically by generating them and ranking them based on relevance.
    """
    # Generate dynamic topics
    topics = generate_topics(input_text)

    # Get embedding for the input text
    input_embedding = get_embedding(input_text)

    # Get embeddings for the generated topics
    topic_embeddings = [get_embedding(topic) for topic in topics]

    # Compute cosine similarity between input and each generated topic
    similarities = cosine_similarity([input_embedding], topic_embeddings)[0]

    # Pair topics with their similarity scores
    related_topics = sorted(zip(topics, similarities), key=lambda x: x[1], reverse=True)

    return related_topics


# input_text = "The importance of renewable energy"
# suggestions = suggest_related_topics(input_text)

# print("Top related topics:")
# for topic, score in suggestions:
#   print(f"- {topic} (Similarity: {score:.2f})")

In [36]:
def mod_bot(text):
    response = client.moderations.create(
        model="text-moderation-latest",
        input=text,
    )

    return(response.results[0].flagged)

# mod_bot('I do not want to hurt anybody')

In [37]:
def header(pdf):
    pdf.set_font("Times", "B", 24)
    pdf.cell(0, 10, "Generated Blog Report", border=0, ln=1, align="C")

def footer(pdf):
    pdf.set_y(-15)
    pdf.set_font("Times", "I", 8)
    pdf.cell(0, 10, f"Page {pdf.page_no()}", align="C")

def add_image(pdf, image_path, x, y, w):
    pdf.set_font("Times", "B", 14)
    pdf.cell(0, 10, "Generated Image", ln=1, align="L")
    pdf.image(image_path, x=x, y=y, w=w)

def add_text(pdf, title, content):
    pdf.set_font("Times", "B", 12)
    pdf.cell(0, 10, title, ln=1, align="L")
    pdf.set_font("Times", size=12)
    pdf.multi_cell(0, 10, content)

def add_topics(pdf, topics):
    pdf.ln(15)
    pdf.set_font("Times", "B", 14)
    pdf.cell(0, 10, "Related Topics", ln=1, align="L")
    pdf.set_font("Times", size=12)
    for topic in topics:
        pdf.cell(0, 10, f"- {topic}", ln=1)

def create_blog_pdf(title, blog_content, image_path, topics, output_file):
    # Converting the image format to JPEG
    with Image.open(image_path) as img:
        img.convert("RGB").save("img.jpg", "JPEG")

    pdf = FPDF()
    pdf.add_page()

    # Add Blog Content
    add_text(pdf, title, blog_content)

    # Add Related Topics
    add_topics(pdf, topics)

    # Add Image
    if image_path:
        pdf.add_page()
        add_image(pdf, image_path, x=10, y=pdf.get_y() + 10, w=180)
        pdf.ln(85)  # Adjust space below image

    # Save PDF
    pdf.output(output_file)
    print(f"PDF saved as {output_file}")


# Example Usage
# title = "The Importance of Renewable Energy"
# blog_content = text_bot(title)
# image_path = img_bot(title, save=True)
# topics = [item[0] for item in suggest_related_topics(title)]
# output_file = "blog_report.pdf"
    
# create_blog_pdf(title, blog_content, image_path, topics, output_file)

In [42]:
def content_bot(subject, text=True, image=True, TTS=True, suggestions=True, save_pdf=True):
    if mod_bot(subject):
        print('The model has classified the subject as harmful. Please try again.')
    
    else:

        if text:
            print("Text bot:")
            text_ = text_bot(subject)
            print(text_)

        if image:
            print('\nImage bot:')
            img_path = img_bot(subject)
            print(img_path)

        if TTS and text:
            print('\nTTS audio file location:')
            print(TTS_bot(text_))

        if suggestions:
            print('\nSuggested related topics:')
            sugg = suggest_related_topics(subject)
            for topic, score in sugg:
                print(f"- {topic} (Similarity: {score:.2f})")

        if save_pdf:
            print('')
            topics = [item[0] for item in suggest_related_topics(subject)]
            create_blog_pdf(subject, text_, img_path, topics, "blog_report.pdf")


content_bot("AI revolutionizing different industries")

Text bot:
Artificial Intelligence (AI) is undeniably revolutionizing various industries with its advanced capabilities and futuristic solutions. In the healthcare sector, AI is enabling powerful diagnostics, personalized treatment plans, and even drug discovery at speeds never before imagined. The manufacturing industry is benefitting from AI's predictive maintenance techniques, optimizing resources, and enhancing operational efficiency. AI-powered chatbots are personalizing customer experiences in the e-commerce industry, increasing customer satisfaction and loyalty. Financial firms are leveraging AI to detect fraudulent activities, predict market trends, and offer personalized financial advice to clients. With the continuous development and integration of AI technology, the potential for innovation across different sectors is limitless, transforming the way industries operate and improving overall productivity and profitability.

Image bot:
C:\Users\Afra\Documents\Neural Networks Not